In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Constantes de base
g = 9.81  # gravité
crr = 0.010  # résistance au roulement
cd = 0.28  # coefficient de traînée du EV9
frontal_area = 2.9  # m²
battery_kWh = 99.8  # batterie utilisable
base_weight = 2650  # poids à vide (kg)

# Widgets
speed_slider = widgets.IntSlider(value=90, min=60, max=130, step=5, description='Vitesse (km/h)')
temp_slider = widgets.FloatSlider(value=22.5, min=-30, max=35, step=2.5, description='Température (°C)')
passenger_slider = widgets.IntSlider(value=4, min=1, max=7, step=1, description='Passagers')
luggage_slider = widgets.IntSlider(value=80, min=0, max=300, step=10, description='Bagages (kg)')
output = widgets.Output()

def estimate_consumption(speed_kmh, temperature, passengers, luggage):
    speed_ms = speed_kmh / 3.6
    passenger_weight = passengers * 70  # adulte/enfant moyen
    total_weight = base_weight + passenger_weight + luggage

    # Densité de l'air selon la température (approx.)
    if temperature <= -20:
        air_density = 1.39
    elif temperature >= 30:
        air_density = 1.16
    else:
        air_density = 1.39 - ((temperature + 20) * (1.39 - 1.16) / 50)

    # Pénalité de chauffage/clim
    if temperature < 5:
        climate_penalty = 0.10  # +10 % chauffage
    elif temperature > 27:
        climate_penalty = 0.05  # +5 % clim
    else:
        climate_penalty = 0

    # Puissances
    aero_power = 0.5 * air_density * cd * frontal_area * speed_ms**3
    rolling_power = total_weight * g * crr * speed_ms
    total_power = aero_power + rolling_power

    # Énergie par km
    time_per_km = 1000 / speed_ms
    energy_per_km_Wh = total_power * time_per_km / 3600
    energy_per_km_Wh *= (1 + climate_penalty)  # pénalité chauffage/clim

    # Résultats
    energy_per_100km_kWh = energy_per_km_Wh * 100 / 1000
    estimated_range_km = battery_kWh / (energy_per_km_Wh / 1000)

    return round(energy_per_km_Wh, 1), round(energy_per_100km_kWh, 2), round(estimated_range_km)

def update(change=None):
    with output:
        output.clear_output()
        e_km, e_100km, autonomy = estimate_consumption(
            speed_slider.value,
            temp_slider.value,
            passenger_slider.value,
            luggage_slider.value
        )
        print(f"🔋 Consommation : {e_100km} kWh / 100 km")
        print(f"🔌 Par km : {e_km} Wh/km")
        print(f"🛣️ Autonomie estimée : {autonomy} km (sur batterie de 99,8 kWh)")

# Relier les widgets
speed_slider.observe(update, names="value")
temp_slider.observe(update, names="value")
passenger_slider.observe(update, names="value")
luggage_slider.observe(update, names="value")

# Affichage
display(speed_slider, temp_slider, passenger_slider, luggage_slider, output)
update()


IntSlider(value=90, description='Vitesse (km/h)', max=130, min=60, step=5)

FloatSlider(value=22.5, description='Température (°C)', max=35.0, min=-30.0, step=2.5)

IntSlider(value=4, description='Passagers', max=7, min=1)

IntSlider(value=80, description='Bagages (kg)', max=300, step=10)

Output()